In [1]:
import os
os.environ['KERAS_BACKEND'] = "cntk"
import sys
import numpy as np
import keras as K
import cntk as C
from keras.applications.resnet50 import ResNet50
from common.params_inf import *
from common.utils import *

Using CNTK backend


In [2]:
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# Faster with channels-last, maybe because model expects that?
K.backend.set_image_data_format('channels_last')

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("Keras: ", K.__version__)
print("CNTK: ", C.__version__)
print("Keras using {}".format(K.backend.backend()))
print("Keras channel ordering is {}".format(K.backend.image_data_format()))
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.14.1
Keras:  2.1.4
CNTK:  2.4
Keras using cntk
Keras channel ordering is channels_last
CUDA Version 8.0.61
CuDNN Version  6.0.21


In [4]:
# Create batches of fake data
fake_input_data_cl, fake_input_data_cf = give_fake_data(BATCH_SIZE*BATCHES_GPU)
print(fake_input_data_cl.shape, fake_input_data_cf.shape)

(1280, 224, 224, 3) (1280, 3, 224, 224)


In [5]:
def predict_fn(classifier, data, batchsize):
    """ Return features from classifier """
    out = np.zeros((len(data), RESNET_FEATURES), np.float32)
    for idx, dta in yield_mb_X(data, batchsize):
        out[idx*batchsize:(idx+1)*batchsize] = classifier.predict_on_batch(dta).squeeze()
    return out

In [6]:
# Download Resnet weights
model = ResNet50(include_top=False, input_shape=(224,224,3))

In [7]:
fake_input_data_cl = np.ascontiguousarray(fake_input_data_cl)
cold_start = predict_fn(model, fake_input_data_cl, BATCH_SIZE)

In [8]:
%%time
features = predict_fn(model, fake_input_data_cl, BATCH_SIZE)

CPU times: user 5.81 s, sys: 1.69 s, total: 7.51 s
Wall time: 7.51 s


In [10]:
print("Images per second {}".format((BATCH_SIZE*BATCHES_GPU)/7.51))

Images per second 170.439414114514
